# 📄 RAG Resume Analysis System

A production-ready Retrieval-Augmented Generation (RAG) system for resume/CV analysis.

## Features
- **Multiple Embedding Models**: text-embedding-3-small, text-embedding-3-large, text-embedding-ada-002 (via OpenRouter)
- **Three Chunking Strategies**: Fixed-length, Semantic, and Recursive
- **Vector Storage**: ChromaDB with persistent storage and A/B testing support
- **Comprehensive Evaluation**: Precision, Recall, F1, MRR, Hit Rate, Latency, Context Relevance
- **Streamlit UI**: Two-page interface for configuration and evaluation

## Quick Start

```bash
# 1. Install dependencies
pip install -r requirements.txt

# 2. Add resume PDFs to data/ folder

# 3. Run Streamlit app
streamlit run app.py
```

## Folder Structure
```
V-4/
├── main.ipynb          # This notebook (all modules)
├── app.py              # Streamlit app (generated)
├── requirements.txt    # Python dependencies
├── data/               # Resume PDFs + metadata
│   ├── metadata.csv    # Per-CV canonical fields
│   └── ground_truth.json  # Query relevance annotations
├── cache/              # Embedding cache (JSON files)
├── outputs/            # Evaluation results (CSV/JSON)
└── chroma_db/          # ChromaDB persistent storage
```

## 1. Setup and Dependencies

In [1]:
# Install dependencies if needed
import subprocess
import sys

def install_package(package):
    """Install a package using pip."""
    subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

# Check and install required packages
required_packages = [
    "chromadb", "openai", "pdfplumber", "numpy", "pandas", 
    "scikit-learn", "nltk", "tqdm", "plotly", "streamlit"
]

print("Checking dependencies...")
for package in required_packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"  ✓ {package}")
    except ImportError:
        print(f"  ⚡ Installing {package}...")
        install_package(package)
        print(f"  ✓ {package} installed")

print("\n✅ All dependencies ready!")

Checking dependencies...
  ✓ chromadb
  ✓ openai
  ✓ pdfplumber
  ✓ numpy
  ✓ pandas
  ⚡ Installing scikit-learn...
  ✓ scikit-learn installed
  ✓ nltk
  ✓ tqdm
  ✓ plotly
  ✓ streamlit

✅ All dependencies ready!


In [2]:
# Import all required libraries
import os
import sys
import json
import hashlib
import time
import re
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Any, Callable
from datetime import datetime
from functools import wraps

# Data processing
import numpy as np
import pandas as pd

# PDF processing
import pdfplumber

# NLP
import nltk
from nltk.tokenize import sent_tokenize

# Vector database
import chromadb
from chromadb.config import Settings

# OpenAI/OpenRouter client
from openai import OpenAI

# Progress bars
from tqdm import tqdm

# Metrics
from sklearn.metrics.pairwise import cosine_similarity

# Download NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Downloading NLTK punkt tokenizer...")
    nltk.download('punkt', quiet=True)

try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab', quiet=True)

print("✅ All imports successful!")

✅ All imports successful!


## 2. Configuration Variables

In [3]:
# ============================================================
# CONFIGURATION - Modify these settings as needed
# ============================================================

# Directory paths (relative to notebook location)
BASE_DIR = Path(".")
DATA_DIR = BASE_DIR / "data"
CACHE_DIR = BASE_DIR / "cache"
OUTPUT_DIR = BASE_DIR / "outputs"
CHROMA_DIR = BASE_DIR / "chroma_db"

# Create directories if they don't exist
for dir_path in [DATA_DIR, CACHE_DIR, OUTPUT_DIR, CHROMA_DIR]:
    dir_path.mkdir(exist_ok=True)

# Available embedding models (OpenRouter compatible)
EMBEDDING_MODELS = [
    "openai/text-embedding-3-small",
    "openai/text-embedding-3-large", 
    "openai/text-embedding-ada-002",
]

# Chunking strategies
CHUNKING_STRATEGIES = ["fixed", "semantic", "recursive"]

# Default chunking parameters
DEFAULT_CHUNK_SIZE = 500
DEFAULT_OVERLAP = 50
DEFAULT_MIN_CHUNK_SIZE = 100

# API settings
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"

# Retrieval settings
DEFAULT_TOP_K = 5

# Batch processing settings
EMBEDDING_BATCH_SIZE = 10
MAX_RETRIES = 3
RETRY_DELAY = 1.0  # seconds

print("📁 Configuration loaded:")
print(f"   Data directory: {DATA_DIR.absolute()}")
print(f"   Cache directory: {CACHE_DIR.absolute()}")
print(f"   Output directory: {OUTPUT_DIR.absolute()}")
print(f"   ChromaDB directory: {CHROMA_DIR.absolute()}")
print(f"\n🤖 Available models: {len(EMBEDDING_MODELS)}")
print(f"📊 Chunking strategies: {CHUNKING_STRATEGIES}")

📁 Configuration loaded:
   Data directory: c:\Users\Mr Computer\Desktop\Folders\CAP\Training\Oppotrain Training\RAG\V-4\data
   Cache directory: c:\Users\Mr Computer\Desktop\Folders\CAP\Training\Oppotrain Training\RAG\V-4\cache
   Output directory: c:\Users\Mr Computer\Desktop\Folders\CAP\Training\Oppotrain Training\RAG\V-4\outputs
   ChromaDB directory: c:\Users\Mr Computer\Desktop\Folders\CAP\Training\Oppotrain Training\RAG\V-4\chroma_db

🤖 Available models: 3
📊 Chunking strategies: ['fixed', 'semantic', 'recursive']


## 3. Base Classes and Helper Functions

### 3.1 PDF Processing

In [4]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extract text content from a PDF file.
    
    Args:
        pdf_path: Path to the PDF file
        
    Returns:
        Extracted text as a string, empty string if extraction fails
    """
    try:
        text_parts = []
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text_parts.append(page_text)
        
        full_text = "\n\n".join(text_parts)
        return full_text.strip()
    
    except Exception as e:
        print(f"⚠️ Error extracting text from {pdf_path}: {e}")
        return ""


def get_pdf_files(directory: Path = DATA_DIR) -> List[Path]:
    """
    Get all PDF files from a directory.
    
    Args:
        directory: Directory to search for PDFs
        
    Returns:
        List of PDF file paths
    """
    pdf_files = list(directory.glob("*.pdf"))
    print(f"📄 Found {len(pdf_files)} PDF files in {directory}")
    return pdf_files


# Test PDF extraction
print("PDF Processing functions defined ✓")

PDF Processing functions defined ✓


### 3.2 Chunking Strategies

In [5]:
def fixed_length_chunking(
    text: str, 
    chunk_size: int = DEFAULT_CHUNK_SIZE, 
    overlap: int = DEFAULT_OVERLAP
) -> List[Dict[str, Any]]:
    """
    Split text into fixed-size chunks with overlap.
    
    Args:
        text: Input text to chunk
        chunk_size: Maximum characters per chunk
        overlap: Number of overlapping characters between chunks
        
    Returns:
        List of chunk dictionaries with text and metadata
    """
    if not text:
        return []
    
    chunks = []
    start = 0
    chunk_idx = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk_text = text[start:end]
        
        chunks.append({
            "text": chunk_text,
            "chunk_index": chunk_idx,
            "start_char": start,
            "end_char": min(end, len(text)),
            "strategy": "fixed"
        })
        
        start += chunk_size - overlap
        chunk_idx += 1
    
    return chunks


def semantic_chunking(
    text: str, 
    target_size: int = DEFAULT_CHUNK_SIZE,
    min_size: int = DEFAULT_MIN_CHUNK_SIZE
) -> List[Dict[str, Any]]:
    """
    Split text at sentence boundaries, combining sentences until target size.
    
    Args:
        text: Input text to chunk
        target_size: Target chunk size in characters
        min_size: Minimum chunk size
        
    Returns:
        List of chunk dictionaries with text and metadata
    """
    if not text:
        return []
    
    # Split into sentences
    try:
        sentences = sent_tokenize(text)
    except Exception:
        # Fallback to simple sentence splitting
        sentences = re.split(r'(?<=[.!?])\s+', text)
    
    chunks = []
    current_chunk = []
    current_length = 0
    chunk_idx = 0
    current_start = 0
    
    for sentence in sentences:
        sentence_length = len(sentence)
        
        # If adding this sentence exceeds target and we have content, save chunk
        if current_length + sentence_length > target_size and current_chunk:
            chunk_text = " ".join(current_chunk)
            chunks.append({
                "text": chunk_text,
                "chunk_index": chunk_idx,
                "start_char": current_start,
                "end_char": current_start + len(chunk_text),
                "strategy": "semantic",
                "sentence_count": len(current_chunk)
            })
            current_start += len(chunk_text) + 1
            chunk_idx += 1
            current_chunk = []
            current_length = 0
        
        current_chunk.append(sentence)
        current_length += sentence_length + 1  # +1 for space
    
    # Don't forget the last chunk
    if current_chunk:
        chunk_text = " ".join(current_chunk)
        chunks.append({
            "text": chunk_text,
            "chunk_index": chunk_idx,
            "start_char": current_start,
            "end_char": current_start + len(chunk_text),
            "strategy": "semantic",
            "sentence_count": len(current_chunk)
        })
    
    return chunks


def recursive_chunking(
    text: str,
    max_chunk_size: int = DEFAULT_CHUNK_SIZE,
    min_chunk_size: int = DEFAULT_MIN_CHUNK_SIZE,
    level: int = 0,
    parent_id: Optional[str] = None
) -> List[Dict[str, Any]]:
    """
    Hierarchically split text: paragraphs -> sentences -> fixed chunks.
    
    Args:
        text: Input text to chunk
        max_chunk_size: Maximum chunk size
        min_chunk_size: Minimum chunk size
        level: Current recursion level (0=paragraphs, 1=sentences, 2=fixed)
        parent_id: ID of parent chunk for tracking hierarchy
        
    Returns:
        List of chunk dictionaries with text and hierarchy metadata
    """
    if not text or len(text) <= max_chunk_size:
        if text:
            return [{
                "text": text,
                "chunk_index": 0,
                "start_char": 0,
                "end_char": len(text),
                "strategy": "recursive",
                "level": level,
                "parent_id": parent_id
            }]
        return []
    
    chunks = []
    
    # Level 0: Split by paragraphs
    if level == 0:
        paragraphs = re.split(r'\n\s*\n', text)
        paragraphs = [p.strip() for p in paragraphs if p.strip()]
    # Level 1: Split by sentences
    elif level == 1:
        try:
            paragraphs = sent_tokenize(text)
        except:
            paragraphs = re.split(r'(?<=[.!?])\s+', text)
    # Level 2: Fixed split
    else:
        # Fall back to fixed chunking
        fixed_chunks = fixed_length_chunking(text, max_chunk_size, overlap=50)
        for i, chunk in enumerate(fixed_chunks):
            chunk["level"] = level
            chunk["parent_id"] = parent_id
            chunk["strategy"] = "recursive"
        return fixed_chunks
    
    current_start = 0
    for i, para in enumerate(paragraphs):
        para_id = f"{parent_id}_{i}" if parent_id else str(i)
        
        if len(para) <= max_chunk_size:
            chunks.append({
                "text": para,
                "chunk_index": len(chunks),
                "start_char": current_start,
                "end_char": current_start + len(para),
                "strategy": "recursive",
                "level": level,
                "parent_id": parent_id,
                "chunk_id": para_id
            })
        else:
            # Recursively split
            sub_chunks = recursive_chunking(
                para, max_chunk_size, min_chunk_size, 
                level=level + 1, parent_id=para_id
            )
            for sub_chunk in sub_chunks:
                sub_chunk["chunk_index"] = len(chunks)
                sub_chunk["start_char"] += current_start
                sub_chunk["end_char"] += current_start
                chunks.append(sub_chunk)
        
        current_start += len(para) + 2  # +2 for paragraph separator
    
    return chunks


def get_chunks(
    text: str, 
    strategy: str = "fixed",
    **kwargs
) -> List[Dict[str, Any]]:
    """
    Dispatcher function to get chunks using the specified strategy.
    
    Args:
        text: Input text to chunk
        strategy: One of 'fixed', 'semantic', 'recursive'
        **kwargs: Additional arguments passed to the chunking function
        
    Returns:
        List of chunk dictionaries
    """
    strategy = strategy.lower()
    
    if strategy == "fixed":
        return fixed_length_chunking(text, **kwargs)
    elif strategy == "semantic":
        return semantic_chunking(text, **kwargs)
    elif strategy == "recursive":
        return recursive_chunking(text, **kwargs)
    else:
        raise ValueError(f"Unknown chunking strategy: {strategy}. Use one of {CHUNKING_STRATEGIES}")


print("Chunking functions defined ✓")
print(f"  • fixed_length_chunking(text, chunk_size={DEFAULT_CHUNK_SIZE}, overlap={DEFAULT_OVERLAP})")
print(f"  • semantic_chunking(text, target_size={DEFAULT_CHUNK_SIZE})")
print(f"  • recursive_chunking(text, max_chunk_size={DEFAULT_CHUNK_SIZE})")
print(f"  • get_chunks(text, strategy='fixed|semantic|recursive')")

Chunking functions defined ✓
  • fixed_length_chunking(text, chunk_size=500, overlap=50)
  • semantic_chunking(text, target_size=500)
  • recursive_chunking(text, max_chunk_size=500)
  • get_chunks(text, strategy='fixed|semantic|recursive')


### 3.3 Embedding Generation with Caching

In [6]:
# ============================================================
# Embedding Cache Functions
# ============================================================

def get_cache_key(text: str, model: str) -> str:
    """
    Generate a unique cache key for text + model combination.
    
    Args:
        text: The text to embed
        model: The embedding model name
        
    Returns:
        SHA256 hash string as cache key
    """
    combined = f"{model}:{text}"
    return hashlib.sha256(combined.encode()).hexdigest()


def get_cache_path(cache_key: str) -> Path:
    """Get the file path for a cache entry."""
    return CACHE_DIR / f"{cache_key}.json"


def load_from_cache(cache_key: str) -> Optional[List[float]]:
    """
    Load embedding from cache if exists.
    
    Args:
        cache_key: The cache key
        
    Returns:
        Cached embedding or None if not found
    """
    cache_path = get_cache_path(cache_key)
    if cache_path.exists():
        try:
            with open(cache_path, 'r') as f:
                data = json.load(f)
                return data.get('embedding')
        except Exception:
            return None
    return None


def save_to_cache(cache_key: str, embedding: List[float], model: str) -> None:
    """
    Save embedding to cache.
    
    Args:
        cache_key: The cache key
        embedding: The embedding vector
        model: The model used (for metadata)
    """
    cache_path = get_cache_path(cache_key)
    try:
        with open(cache_path, 'w') as f:
            json.dump({
                'embedding': embedding,
                'model': model,
                'timestamp': datetime.now().isoformat()
            }, f)
    except Exception as e:
        print(f"⚠️ Failed to save cache: {e}")


def clear_cache() -> int:
    """Clear all cached embeddings. Returns number of files deleted."""
    count = 0
    for cache_file in CACHE_DIR.glob("*.json"):
        cache_file.unlink()
        count += 1
    return count


print("Cache functions defined ✓")

Cache functions defined ✓


In [7]:
# ============================================================
# API Key Validation and Embedding Generation
# ============================================================

def validate_openrouter_key(api_key: str) -> Tuple[bool, str]:
    """
    Validate OpenRouter API key by making a test request.
    
    Args:
        api_key: The OpenRouter API key
        
    Returns:
        Tuple of (is_valid, message)
    """
    if not api_key or len(api_key) < 10:
        return False, "API key is too short or empty"
    
    try:
        client = OpenAI(
            base_url=OPENROUTER_BASE_URL,
            api_key=api_key
        )
        
        # Make a minimal embedding request to validate
        response = client.embeddings.create(
            model="openai/text-embedding-3-small",
            input="test"
        )
        
        if response.data and len(response.data) > 0:
            return True, "API key is valid ✓"
        else:
            return False, "API returned empty response"
            
    except Exception as e:
        error_msg = str(e)
        if "401" in error_msg or "unauthorized" in error_msg.lower():
            return False, "Invalid API key (401 Unauthorized)"
        elif "429" in error_msg:
            return True, "API key valid (rate limited - try again later)"
        else:
            return False, f"API error: {error_msg[:100]}"


def get_embedding(
    text: str, 
    model: str, 
    api_key: str,
    use_cache: bool = True
) -> List[float]:
    """
    Get embedding for text using OpenRouter API.
    
    Args:
        text: Text to embed
        model: Embedding model name (e.g., 'openai/text-embedding-3-small')
        api_key: OpenRouter API key
        use_cache: Whether to use caching
        
    Returns:
        Embedding vector as list of floats
    """
    # Check cache first
    if use_cache:
        cache_key = get_cache_key(text, model)
        cached = load_from_cache(cache_key)
        if cached is not None:
            return cached
    
    # Make API request with retry
    client = OpenAI(
        base_url=OPENROUTER_BASE_URL,
        api_key=api_key
    )
    
    for attempt in range(MAX_RETRIES):
        try:
            response = client.embeddings.create(
                model=model,
                input=text
            )
            
            embedding = response.data[0].embedding
            
            # Save to cache
            if use_cache:
                save_to_cache(cache_key, embedding, model)
            
            return embedding
            
        except Exception as e:
            if attempt < MAX_RETRIES - 1:
                wait_time = RETRY_DELAY * (2 ** attempt)  # Exponential backoff
                time.sleep(wait_time)
            else:
                raise RuntimeError(f"Failed to get embedding after {MAX_RETRIES} attempts: {e}")


def get_embeddings_batch(
    texts: List[str],
    model: str,
    api_key: str,
    use_cache: bool = True,
    show_progress: bool = True
) -> List[List[float]]:
    """
    Get embeddings for multiple texts with batching and progress bar.
    
    Args:
        texts: List of texts to embed
        model: Embedding model name
        api_key: OpenRouter API key
        use_cache: Whether to use caching
        show_progress: Whether to show progress bar
        
    Returns:
        List of embedding vectors
    """
    embeddings = []
    
    iterator = tqdm(texts, desc="Generating embeddings") if show_progress else texts
    
    for text in iterator:
        embedding = get_embedding(text, model, api_key, use_cache)
        embeddings.append(embedding)
    
    return embeddings


print("Embedding functions defined ✓")
print("  • validate_openrouter_key(api_key)")
print("  • get_embedding(text, model, api_key)")
print("  • get_embeddings_batch(texts, model, api_key)")

Embedding functions defined ✓
  • validate_openrouter_key(api_key)
  • get_embedding(text, model, api_key)
  • get_embeddings_batch(texts, model, api_key)


### 3.4 ChromaDB Operations

In [8]:
# ============================================================
# ChromaDB Vector Database Operations
# ============================================================

# Global ChromaDB client (persistent)
_chroma_client = None

def get_chroma_client() -> chromadb.PersistentClient:
    """Get or create the ChromaDB persistent client."""
    global _chroma_client
    if _chroma_client is None:
        _chroma_client = chromadb.PersistentClient(path=str(CHROMA_DIR))
    return _chroma_client


def initialize_chromadb(collection_name: str) -> chromadb.Collection:
    """
    Initialize or get a ChromaDB collection.
    
    Args:
        collection_name: Name of the collection
        
    Returns:
        ChromaDB Collection object
    """
    client = get_chroma_client()
    collection = client.get_or_create_collection(
        name=collection_name,
        metadata={"hnsw:space": "cosine"}  # Use cosine similarity
    )
    return collection


def get_collection_name(model: str, strategy: str) -> str:
    """
    Generate a collection name from model and strategy for A/B testing.
    
    Args:
        model: Embedding model name
        strategy: Chunking strategy
        
    Returns:
        Sanitized collection name
    """
    # Sanitize model name (remove slashes, etc.)
    model_clean = model.replace("/", "_").replace("-", "_")
    return f"{model_clean}_{strategy}"


def index_documents(
    chunks: List[Dict[str, Any]],
    embeddings: List[List[float]],
    source_file: str,
    collection: chromadb.Collection,
    embedding_model: str,
    chunking_strategy: str
) -> int:
    """
    Index document chunks into ChromaDB.
    
    Args:
        chunks: List of chunk dictionaries with text and metadata
        embeddings: List of embedding vectors
        source_file: Source PDF filename
        collection: ChromaDB collection
        embedding_model: Name of embedding model used
        chunking_strategy: Name of chunking strategy used
        
    Returns:
        Number of chunks indexed
    """
    if not chunks or not embeddings:
        return 0
    
    ids = []
    documents = []
    metadatas = []
    
    for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        chunk_id = f"{source_file}_{chunking_strategy}_{i}"
        
        ids.append(chunk_id)
        documents.append(chunk["text"])
        
        metadata = {
            "source_file": source_file,
            "chunk_index": chunk.get("chunk_index", i),
            "start_char": chunk.get("start_char", 0),
            "end_char": chunk.get("end_char", len(chunk["text"])),
            "chunking_strategy": chunking_strategy,
            "embedding_model": embedding_model,
            "indexed_at": datetime.now().isoformat()
        }
        
        # Add any extra metadata from chunk
        for key in ["level", "parent_id", "sentence_count"]:
            if key in chunk:
                metadata[key] = str(chunk[key])
        
        metadatas.append(metadata)
    
    # Add to collection
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=embeddings,
        metadatas=metadatas
    )
    
    return len(ids)


def retrieve_similar_chunks(
    query: str,
    collection: chromadb.Collection,
    embedding_model: str,
    api_key: str,
    top_k: int = DEFAULT_TOP_K
) -> List[Dict[str, Any]]:
    """
    Retrieve similar chunks for a query.
    
    Args:
        query: Query text
        collection: ChromaDB collection
        embedding_model: Model to use for query embedding
        api_key: OpenRouter API key
        top_k: Number of results to return
        
    Returns:
        List of result dictionaries with text, metadata, and distance
    """
    # Get query embedding
    query_embedding = get_embedding(query, embedding_model, api_key)
    
    # Query collection
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
        include=["documents", "metadatas", "distances"]
    )
    
    # Format results
    formatted_results = []
    if results and results['ids'] and len(results['ids']) > 0:
        for i in range(len(results['ids'][0])):
            formatted_results.append({
                "chunk_id": results['ids'][0][i],
                "text": results['documents'][0][i] if results['documents'] else "",
                "metadata": results['metadatas'][0][i] if results['metadatas'] else {},
                "distance": results['distances'][0][i] if results['distances'] else 0,
                "similarity": 1 - results['distances'][0][i] if results['distances'] else 1
            })
    
    return formatted_results


def list_collections() -> List[str]:
    """List all collections in ChromaDB."""
    client = get_chroma_client()
    collections = client.list_collections()
    return [c.name for c in collections]


def delete_collection(collection_name: str) -> bool:
    """
    Delete a collection from ChromaDB.
    
    Args:
        collection_name: Name of collection to delete
        
    Returns:
        True if deleted successfully
    """
    try:
        client = get_chroma_client()
        client.delete_collection(collection_name)
        return True
    except Exception as e:
        print(f"⚠️ Failed to delete collection: {e}")
        return False


def get_collection_stats(collection_name: str) -> Dict[str, Any]:
    """
    Get statistics for a collection.
    
    Args:
        collection_name: Name of collection
        
    Returns:
        Dictionary with collection statistics
    """
    try:
        client = get_chroma_client()
        collection = client.get_collection(collection_name)
        count = collection.count()
        
        # Get sample metadata
        sample = collection.peek(limit=1)
        
        return {
            "name": collection_name,
            "count": count,
            "sample_metadata": sample.get("metadatas", [{}])[0] if sample else {}
        }
    except Exception as e:
        return {"name": collection_name, "error": str(e)}


print("ChromaDB functions defined ✓")
print("  • initialize_chromadb(collection_name)")
print("  • index_documents(chunks, embeddings, source_file, collection, ...)")
print("  • retrieve_similar_chunks(query, collection, embedding_model, api_key, top_k)")
print("  • list_collections(), delete_collection(), get_collection_stats()")

ChromaDB functions defined ✓
  • initialize_chromadb(collection_name)
  • index_documents(chunks, embeddings, source_file, collection, ...)
  • retrieve_similar_chunks(query, collection, embedding_model, api_key, top_k)
  • list_collections(), delete_collection(), get_collection_stats()


### 3.5 Evaluation Metrics

In [9]:
# ============================================================
# Evaluation Metrics
# ============================================================

def calculate_precision(retrieved: List[str], relevant: List[str]) -> float:
    """
    Calculate precision: proportion of retrieved items that are relevant.
    
    Args:
        retrieved: List of retrieved item IDs/texts
        relevant: List of relevant item IDs/texts
        
    Returns:
        Precision score (0.0 to 1.0)
    """
    if not retrieved:
        return 0.0
    
    retrieved_set = set(retrieved)
    relevant_set = set(relevant)
    
    true_positives = len(retrieved_set & relevant_set)
    return true_positives / len(retrieved)


def calculate_recall(retrieved: List[str], relevant: List[str]) -> float:
    """
    Calculate recall: proportion of relevant items that were retrieved.
    
    Args:
        retrieved: List of retrieved item IDs/texts
        relevant: List of relevant item IDs/texts
        
    Returns:
        Recall score (0.0 to 1.0)
    """
    if not relevant:
        return 0.0
    
    retrieved_set = set(retrieved)
    relevant_set = set(relevant)
    
    true_positives = len(retrieved_set & relevant_set)
    return true_positives / len(relevant)


def calculate_f1_score(precision: float, recall: float) -> float:
    """
    Calculate F1 score: harmonic mean of precision and recall.
    
    Args:
        precision: Precision score
        recall: Recall score
        
    Returns:
        F1 score (0.0 to 1.0)
    """
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)


def calculate_mrr(ranked_results: List[List[str]], relevant: List[str]) -> float:
    """
    Calculate Mean Reciprocal Rank.
    
    Args:
        ranked_results: List of ranked result lists (one per query)
        relevant: List of relevant item IDs
        
    Returns:
        MRR score (0.0 to 1.0)
    """
    if not ranked_results:
        return 0.0
    
    relevant_set = set(relevant)
    reciprocal_ranks = []
    
    for results in ranked_results:
        for rank, item in enumerate(results, 1):
            if item in relevant_set:
                reciprocal_ranks.append(1.0 / rank)
                break
        else:
            reciprocal_ranks.append(0.0)
    
    return sum(reciprocal_ranks) / len(reciprocal_ranks)


def calculate_hit_rate(results: List[Dict[str, Any]], relevant: List[str]) -> float:
    """
    Calculate hit rate: proportion of queries with at least one relevant result.
    
    Args:
        results: List of result dictionaries
        relevant: List of relevant item IDs/texts
        
    Returns:
        Hit rate (0.0 to 1.0)
    """
    if not results:
        return 0.0
    
    relevant_set = set(relevant)
    
    # Check if any retrieved item is relevant
    for result in results:
        chunk_id = result.get("chunk_id", "")
        text = result.get("text", "")
        
        if chunk_id in relevant_set or text in relevant_set:
            return 1.0
        
        # Also check if any relevant item is a substring of retrieved text
        for rel_item in relevant_set:
            if rel_item.lower() in text.lower():
                return 1.0
    
    return 0.0


def measure_latency(func: Callable, *args, **kwargs) -> Tuple[Any, float]:
    """
    Measure execution time of a function.
    
    Args:
        func: Function to measure
        *args: Positional arguments for the function
        **kwargs: Keyword arguments for the function
        
    Returns:
        Tuple of (function result, latency in milliseconds)
    """
    start_time = time.perf_counter()
    result = func(*args, **kwargs)
    end_time = time.perf_counter()
    
    latency_ms = (end_time - start_time) * 1000
    return result, latency_ms


def calculate_context_relevance(
    query_embedding: List[float],
    retrieved_embeddings: List[List[float]]
) -> float:
    """
    Calculate average cosine similarity between query and retrieved chunks.
    
    Args:
        query_embedding: Query embedding vector
        retrieved_embeddings: List of retrieved chunk embeddings
        
    Returns:
        Average similarity score (0.0 to 1.0)
    """
    if not retrieved_embeddings:
        return 0.0
    
    query_arr = np.array(query_embedding).reshape(1, -1)
    retrieved_arr = np.array(retrieved_embeddings)
    
    similarities = cosine_similarity(query_arr, retrieved_arr)[0]
    return float(np.mean(similarities))


print("Evaluation metrics defined ✓")
print("  • calculate_precision(retrieved, relevant)")
print("  • calculate_recall(retrieved, relevant)")
print("  • calculate_f1_score(precision, recall)")
print("  • calculate_mrr(ranked_results, relevant)")
print("  • calculate_hit_rate(results, relevant)")
print("  • measure_latency(func, *args)")
print("  • calculate_context_relevance(query_embedding, retrieved_embeddings)")

Evaluation metrics defined ✓
  • calculate_precision(retrieved, relevant)
  • calculate_recall(retrieved, relevant)
  • calculate_f1_score(precision, recall)
  • calculate_mrr(ranked_results, relevant)
  • calculate_hit_rate(results, relevant)
  • measure_latency(func, *args)
  • calculate_context_relevance(query_embedding, retrieved_embeddings)


### 3.6 Ground Truth Management

In [10]:
# ============================================================
# Ground Truth and Metadata Management
# ============================================================

def load_ground_truth(filepath: str = None) -> Dict[str, Any]:
    """
    Load ground truth annotations from JSON file.
    
    Args:
        filepath: Path to ground truth JSON file
        
    Returns:
        Dictionary of query -> relevant items mapping
    """
    if filepath is None:
        filepath = DATA_DIR / "ground_truth.json"
    else:
        filepath = Path(filepath)
    
    if not filepath.exists():
        print(f"⚠️ Ground truth file not found: {filepath}")
        return {}
    
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        print(f"⚠️ Error loading ground truth: {e}")
        return {}


def load_metadata(filepath: str = None) -> pd.DataFrame:
    """
    Load resume metadata from CSV file.
    
    Args:
        filepath: Path to metadata CSV file
        
    Returns:
        DataFrame with resume metadata
    """
    if filepath is None:
        filepath = DATA_DIR / "metadata.csv"
    else:
        filepath = Path(filepath)
    
    if not filepath.exists():
        print(f"⚠️ Metadata file not found: {filepath}")
        return pd.DataFrame()
    
    try:
        return pd.read_csv(filepath)
    except Exception as e:
        print(f"⚠️ Error loading metadata: {e}")
        return pd.DataFrame()


def map_canonical_to_chunks(
    canonical_answers: List[str],
    chunks: List[Dict[str, Any]],
    fuzzy_threshold: float = 0.8
) -> List[str]:
    """
    Map canonical answers to chunk IDs using exact and fuzzy matching.
    
    Args:
        canonical_answers: List of canonical answer strings
        chunks: List of chunk dictionaries
        fuzzy_threshold: Minimum similarity for fuzzy match (0-1)
        
    Returns:
        List of matching chunk IDs
    """
    matching_chunk_ids = []
    
    for chunk in chunks:
        chunk_text = chunk.get("text", "").lower()
        chunk_id = chunk.get("chunk_id", str(chunk.get("chunk_index", "")))
        
        for answer in canonical_answers:
            answer_lower = answer.lower()
            
            # Exact substring match
            if answer_lower in chunk_text:
                matching_chunk_ids.append(chunk_id)
                break
            
            # Simple fuzzy match: check if most words from answer are in chunk
            answer_words = set(answer_lower.split())
            chunk_words = set(chunk_text.split())
            
            if answer_words and len(answer_words & chunk_words) / len(answer_words) >= fuzzy_threshold:
                matching_chunk_ids.append(chunk_id)
                break
    
    return matching_chunk_ids


def get_relevant_chunks_for_query(
    query_id: str,
    ground_truth: Dict[str, Any],
    all_chunks: List[Dict[str, Any]]
) -> List[str]:
    """
    Get relevant chunk IDs for a query based on ground truth.
    
    Args:
        query_id: The query identifier
        ground_truth: Ground truth dictionary
        all_chunks: All indexed chunks
        
    Returns:
        List of relevant chunk IDs
    """
    if query_id not in ground_truth:
        return []
    
    query_gt = ground_truth[query_id]
    relevant = query_gt.get("relevant", [])
    
    # Extract canonical values from ground truth
    canonical_values = []
    for item in relevant:
        if isinstance(item, dict):
            value = item.get("value", "")
            if value and item.get("type") != "not_relevant":
                canonical_values.append(value)
        elif isinstance(item, str):
            canonical_values.append(item)
    
    if canonical_values:
        return map_canonical_to_chunks(canonical_values, all_chunks)
    
    return []


print("Ground truth functions defined ✓")
print("  • load_ground_truth(filepath)")
print("  • load_metadata(filepath)")
print("  • map_canonical_to_chunks(canonical_answers, chunks)")
print("  • get_relevant_chunks_for_query(query_id, ground_truth, all_chunks)")

Ground truth functions defined ✓
  • load_ground_truth(filepath)
  • load_metadata(filepath)
  • map_canonical_to_chunks(canonical_answers, chunks)
  • get_relevant_chunks_for_query(query_id, ground_truth, all_chunks)


## 4. Main Pipelines

### 4.1 Resume Processing Pipeline

In [11]:
# ============================================================
# Main Processing Pipeline
# ============================================================

def process_resumes(
    pdf_files: List[Path],
    chunking_strategy: str,
    embedding_model: str,
    api_key: str,
    collection_name: str = None,
    chunk_size: int = DEFAULT_CHUNK_SIZE,
    overlap: int = DEFAULT_OVERLAP,
    show_progress: bool = True
) -> Dict[str, Any]:
    """
    Process resume PDFs: extract text, chunk, embed, and index.
    
    Args:
        pdf_files: List of PDF file paths
        chunking_strategy: One of 'fixed', 'semantic', 'recursive'
        embedding_model: Embedding model name
        api_key: OpenRouter API key
        collection_name: ChromaDB collection name (auto-generated if None)
        chunk_size: Size for chunking
        overlap: Overlap for fixed chunking
        show_progress: Show progress bars
        
    Returns:
        Summary dictionary with processing statistics
    """
    if not pdf_files:
        return {"error": "No PDF files provided", "processed": 0}
    
    # Generate collection name if not provided
    if collection_name is None:
        collection_name = get_collection_name(embedding_model, chunking_strategy)
    
    # Initialize collection
    collection = initialize_chromadb(collection_name)
    
    # Processing stats
    stats = {
        "collection_name": collection_name,
        "embedding_model": embedding_model,
        "chunking_strategy": chunking_strategy,
        "files_processed": 0,
        "files_failed": 0,
        "total_chunks": 0,
        "total_characters": 0,
        "processing_time_ms": 0
    }
    
    start_time = time.perf_counter()
    
    # Process each PDF
    iterator = tqdm(pdf_files, desc="Processing PDFs") if show_progress else pdf_files
    
    for pdf_path in iterator:
        try:
            # Extract text
            text = extract_text_from_pdf(str(pdf_path))
            
            if not text:
                print(f"⚠️ No text extracted from {pdf_path.name}")
                stats["files_failed"] += 1
                continue
            
            # Chunk text
            if chunking_strategy == "fixed":
                chunks = fixed_length_chunking(text, chunk_size, overlap)
            elif chunking_strategy == "semantic":
                chunks = semantic_chunking(text, chunk_size)
            elif chunking_strategy == "recursive":
                chunks = recursive_chunking(text, chunk_size)
            else:
                chunks = fixed_length_chunking(text, chunk_size, overlap)
            
            if not chunks:
                print(f"⚠️ No chunks created from {pdf_path.name}")
                stats["files_failed"] += 1
                continue
            
            # Get embeddings
            chunk_texts = [c["text"] for c in chunks]
            embeddings = get_embeddings_batch(
                chunk_texts, embedding_model, api_key, 
                show_progress=False
            )
            
            # Index documents
            indexed_count = index_documents(
                chunks, embeddings, pdf_path.name,
                collection, embedding_model, chunking_strategy
            )
            
            stats["files_processed"] += 1
            stats["total_chunks"] += indexed_count
            stats["total_characters"] += len(text)
            
        except Exception as e:
            print(f"⚠️ Error processing {pdf_path.name}: {e}")
            stats["files_failed"] += 1
    
    end_time = time.perf_counter()
    stats["processing_time_ms"] = (end_time - start_time) * 1000
    
    print(f"\n✅ Processing complete!")
    print(f"   Files processed: {stats['files_processed']}")
    print(f"   Total chunks indexed: {stats['total_chunks']}")
    print(f"   Collection: {collection_name}")
    
    return stats


print("Processing pipeline defined ✓")
print("  • process_resumes(pdf_files, chunking_strategy, embedding_model, api_key, ...)")

Processing pipeline defined ✓
  • process_resumes(pdf_files, chunking_strategy, embedding_model, api_key, ...)


### 4.2 Evaluation Pipeline

In [12]:
# ============================================================
# Evaluation Pipeline
# ============================================================

def evaluate_retrieval(
    queries: List[str],
    collection_name: str,
    embedding_model: str,
    api_key: str,
    ground_truth: Dict[str, Any] = None,
    top_k: int = DEFAULT_TOP_K,
    show_progress: bool = True
) -> Dict[str, Any]:
    """
    Evaluate retrieval performance across queries.
    
    Args:
        queries: List of query strings
        collection_name: ChromaDB collection name
        embedding_model: Model for query embedding
        api_key: OpenRouter API key
        ground_truth: Ground truth dictionary (optional)
        top_k: Number of results per query
        show_progress: Show progress bars
        
    Returns:
        Comprehensive evaluation results dictionary
    """
    if not queries:
        return {"error": "No queries provided"}
    
    # Initialize collection
    collection = initialize_chromadb(collection_name)
    
    # Results storage
    all_results = []
    per_query_metrics = []
    total_latency_ms = 0
    
    iterator = tqdm(queries, desc="Evaluating queries") if show_progress else queries
    
    for i, query in enumerate(iterator):
        query_id = f"q_{i}"
        
        # Retrieve with timing
        results, latency_ms = measure_latency(
            retrieve_similar_chunks,
            query, collection, embedding_model, api_key, top_k
        )
        
        total_latency_ms += latency_ms
        
        # Get retrieved chunk IDs
        retrieved_ids = [r.get("chunk_id", "") for r in results]
        retrieved_texts = [r.get("text", "") for r in results]
        
        # Get relevant items from ground truth if available
        relevant_ids = []
        if ground_truth and query_id in ground_truth:
            gt_entry = ground_truth[query_id]
            relevant = gt_entry.get("relevant", [])
            for item in relevant:
                if isinstance(item, dict) and item.get("type") != "not_relevant":
                    relevant_ids.append(item.get("value", ""))
                elif isinstance(item, str):
                    relevant_ids.append(item)
        
        # Calculate metrics
        precision = calculate_precision(retrieved_ids, relevant_ids) if relevant_ids else 0
        recall = calculate_recall(retrieved_ids, relevant_ids) if relevant_ids else 0
        f1 = calculate_f1_score(precision, recall)
        hit_rate = calculate_hit_rate(results, relevant_ids) if relevant_ids else 0
        
        # Context relevance (average similarity of top results)
        avg_similarity = np.mean([r.get("similarity", 0) for r in results]) if results else 0
        
        query_result = {
            "query_id": query_id,
            "query": query,
            "retrieved_count": len(results),
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "hit_rate": hit_rate,
            "context_relevance": avg_similarity,
            "latency_ms": latency_ms,
            "top_results": results[:3]  # Store top 3 for display
        }
        
        per_query_metrics.append(query_result)
        all_results.append({
            "query": query,
            "results": results
        })
    
    # Aggregate metrics
    n_queries = len(queries)
    
    aggregated = {
        "collection_name": collection_name,
        "embedding_model": embedding_model,
        "top_k": top_k,
        "num_queries": n_queries,
        "avg_precision": np.mean([m["precision"] for m in per_query_metrics]),
        "avg_recall": np.mean([m["recall"] for m in per_query_metrics]),
        "avg_f1": np.mean([m["f1"] for m in per_query_metrics]),
        "avg_hit_rate": np.mean([m["hit_rate"] for m in per_query_metrics]),
        "avg_context_relevance": np.mean([m["context_relevance"] for m in per_query_metrics]),
        "avg_latency_ms": total_latency_ms / n_queries if n_queries > 0 else 0,
        "total_latency_ms": total_latency_ms,
        "per_query_metrics": per_query_metrics,
        "all_results": all_results
    }
    
    print(f"\n📊 Evaluation Results:")
    print(f"   Queries evaluated: {n_queries}")
    print(f"   Avg Precision: {aggregated['avg_precision']:.3f}")
    print(f"   Avg Recall: {aggregated['avg_recall']:.3f}")
    print(f"   Avg F1: {aggregated['avg_f1']:.3f}")
    print(f"   Avg Latency: {aggregated['avg_latency_ms']:.2f} ms")
    
    return aggregated


def compare_configurations(
    queries: List[str],
    configurations: List[Dict[str, str]],
    api_key: str,
    ground_truth: Dict[str, Any] = None,
    top_k: int = DEFAULT_TOP_K
) -> pd.DataFrame:
    """
    Compare multiple model+strategy configurations.
    
    Args:
        queries: List of test queries
        configurations: List of dicts with 'model' and 'strategy' keys
        api_key: OpenRouter API key
        ground_truth: Ground truth dictionary
        top_k: Number of results per query
        
    Returns:
        DataFrame comparing all configurations
    """
    results = []
    
    for config in configurations:
        model = config["model"]
        strategy = config["strategy"]
        collection_name = get_collection_name(model, strategy)
        
        print(f"\n🔍 Evaluating: {model} + {strategy}")
        
        try:
            eval_result = evaluate_retrieval(
                queries, collection_name, model, api_key,
                ground_truth, top_k, show_progress=False
            )
            
            results.append({
                "Model": model,
                "Strategy": strategy,
                "Precision": eval_result["avg_precision"],
                "Recall": eval_result["avg_recall"],
                "F1": eval_result["avg_f1"],
                "Hit Rate": eval_result["avg_hit_rate"],
                "Context Relevance": eval_result["avg_context_relevance"],
                "Latency (ms)": eval_result["avg_latency_ms"]
            })
        except Exception as e:
            print(f"  ⚠️ Error: {e}")
            results.append({
                "Model": model,
                "Strategy": strategy,
                "Precision": 0,
                "Recall": 0,
                "F1": 0,
                "Hit Rate": 0,
                "Context Relevance": 0,
                "Latency (ms)": 0,
                "Error": str(e)
            })
    
    return pd.DataFrame(results)


print("Evaluation pipeline defined ✓")
print("  • evaluate_retrieval(queries, collection_name, embedding_model, api_key, ...)")
print("  • compare_configurations(queries, configurations, api_key, ...)")

Evaluation pipeline defined ✓
  • evaluate_retrieval(queries, collection_name, embedding_model, api_key, ...)
  • compare_configurations(queries, configurations, api_key, ...)


### 4.3 Results Export

In [13]:
# ============================================================
# Results Export Functions
# ============================================================

def export_results_csv(results: Dict[str, Any], filename: str = None) -> str:
    """
    Export evaluation results to CSV.
    
    Args:
        results: Evaluation results dictionary
        filename: Output filename (auto-generated if None)
        
    Returns:
        Path to saved file
    """
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"evaluation_results_{timestamp}.csv"
    
    filepath = OUTPUT_DIR / filename
    
    # Convert per-query metrics to DataFrame
    if "per_query_metrics" in results:
        df = pd.DataFrame(results["per_query_metrics"])
        # Remove complex nested columns
        columns_to_keep = [c for c in df.columns if c != "top_results"]
        df = df[columns_to_keep]
        df.to_csv(filepath, index=False)
    else:
        # Export aggregated metrics
        df = pd.DataFrame([{
            k: v for k, v in results.items() 
            if not isinstance(v, (list, dict))
        }])
        df.to_csv(filepath, index=False)
    
    print(f"📁 Results exported to: {filepath}")
    return str(filepath)


def export_results_json(results: Dict[str, Any], filename: str = None) -> str:
    """
    Export evaluation results to JSON.
    
    Args:
        results: Evaluation results dictionary
        filename: Output filename (auto-generated if None)
        
    Returns:
        Path to saved file
    """
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"evaluation_results_{timestamp}.json"
    
    filepath = OUTPUT_DIR / filename
    
    # Make results JSON serializable
    serializable = {}
    for key, value in results.items():
        if isinstance(value, (np.floating, np.integer)):
            serializable[key] = float(value)
        elif isinstance(value, np.ndarray):
            serializable[key] = value.tolist()
        else:
            serializable[key] = value
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(serializable, f, indent=2, default=str)
    
    print(f"📁 Results exported to: {filepath}")
    return str(filepath)


def export_comparison_report(
    comparison_df: pd.DataFrame,
    filename: str = None
) -> str:
    """
    Export comparison DataFrame to CSV with summary.
    
    Args:
        comparison_df: DataFrame from compare_configurations
        filename: Output filename
        
    Returns:
        Path to saved file
    """
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"comparison_report_{timestamp}.csv"
    
    filepath = OUTPUT_DIR / filename
    comparison_df.to_csv(filepath, index=False)
    
    print(f"📁 Comparison report exported to: {filepath}")
    return str(filepath)


print("Export functions defined ✓")
print("  • export_results_csv(results, filename)")
print("  • export_results_json(results, filename)")
print("  • export_comparison_report(comparison_df, filename)")

Export functions defined ✓
  • export_results_csv(results, filename)
  • export_results_json(results, filename)
  • export_comparison_report(comparison_df, filename)


## 5. Streamlit Application

Run the cell below to generate `app.py`, then launch with:
```bash
streamlit run app.py
```

In [14]:
%%writefile app.py
"""
RAG Resume Analysis - Streamlit Application
Two-page UI for configuration/upload and evaluation/testing
"""

import streamlit as st
import os
import json
import hashlib
import time
import re
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime

import numpy as np
import pandas as pd
import pdfplumber
import chromadb
from openai import OpenAI
import plotly.express as px
import plotly.graph_objects as go

# Try to import NLTK
try:
    import nltk
    from nltk.tokenize import sent_tokenize
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt', quiet=True)
    try:
        nltk.data.find('tokenizers/punkt_tab')
    except LookupError:
        nltk.download('punkt_tab', quiet=True)
    NLTK_AVAILABLE = True
except:
    NLTK_AVAILABLE = False

# ============================================================
# Configuration
# ============================================================
BASE_DIR = Path(".")
DATA_DIR = BASE_DIR / "data"
CACHE_DIR = BASE_DIR / "cache"
OUTPUT_DIR = BASE_DIR / "outputs"
CHROMA_DIR = BASE_DIR / "chroma_db"

for dir_path in [DATA_DIR, CACHE_DIR, OUTPUT_DIR, CHROMA_DIR]:
    dir_path.mkdir(exist_ok=True)

EMBEDDING_MODELS = [
    "openai/text-embedding-3-small",
    "openai/text-embedding-3-large",
    "openai/text-embedding-ada-002",
]

CHUNKING_STRATEGIES = ["fixed", "semantic", "recursive"]
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
DEFAULT_CHUNK_SIZE = 500
DEFAULT_OVERLAP = 50
DEFAULT_TOP_K = 5
MAX_RETRIES = 3
RETRY_DELAY = 1.0

# ============================================================
# Helper Functions (copied from notebook for standalone app)
# ============================================================

def get_cache_key(text: str, model: str) -> str:
    combined = f"{model}:{text}"
    return hashlib.sha256(combined.encode()).hexdigest()

def get_cache_path(cache_key: str) -> Path:
    return CACHE_DIR / f"{cache_key}.json"

def load_from_cache(cache_key: str) -> Optional[List[float]]:
    cache_path = get_cache_path(cache_key)
    if cache_path.exists():
        try:
            with open(cache_path, 'r') as f:
                return json.load(f).get('embedding')
        except:
            return None
    return None

def save_to_cache(cache_key: str, embedding: List[float], model: str):
    try:
        with open(get_cache_path(cache_key), 'w') as f:
            json.dump({'embedding': embedding, 'model': model, 'timestamp': datetime.now().isoformat()}, f)
    except:
        pass

def validate_openrouter_key(api_key: str) -> Tuple[bool, str]:
    if not api_key or len(api_key) < 10:
        return False, "API key is too short or empty"
    try:
        client = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=api_key)
        response = client.embeddings.create(model="openai/text-embedding-3-small", input="test")
        if response.data:
            return True, "API key is valid ✓"
        return False, "API returned empty response"
    except Exception as e:
        error_msg = str(e)
        if "401" in error_msg:
            return False, "Invalid API key"
        elif "429" in error_msg:
            return True, "API key valid (rate limited)"
        return False, f"Error: {error_msg[:100]}"

def get_embedding(text: str, model: str, api_key: str, use_cache: bool = True) -> List[float]:
    if use_cache:
        cache_key = get_cache_key(text, model)
        cached = load_from_cache(cache_key)
        if cached:
            return cached
    
    client = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=api_key)
    for attempt in range(MAX_RETRIES):
        try:
            response = client.embeddings.create(model=model, input=text)
            embedding = response.data[0].embedding
            if use_cache:
                save_to_cache(cache_key, embedding, model)
            return embedding
        except Exception as e:
            if attempt < MAX_RETRIES - 1:
                time.sleep(RETRY_DELAY * (2 ** attempt))
            else:
                raise RuntimeError(f"Failed after {MAX_RETRIES} attempts: {e}")

def extract_text_from_pdf(pdf_path: str) -> str:
    try:
        text_parts = []
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text_parts.append(page_text)
        return "\n\n".join(text_parts).strip()
    except Exception as e:
        st.error(f"Error extracting PDF: {e}")
        return ""

def fixed_length_chunking(text: str, chunk_size: int = 500, overlap: int = 50) -> List[Dict]:
    if not text:
        return []
    chunks = []
    start = 0
    idx = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append({"text": text[start:end], "chunk_index": idx, "start_char": start, "end_char": min(end, len(text)), "strategy": "fixed"})
        start += chunk_size - overlap
        idx += 1
    return chunks

def semantic_chunking(text: str, target_size: int = 500) -> List[Dict]:
    if not text:
        return []
    if NLTK_AVAILABLE:
        try:
            sentences = sent_tokenize(text)
        except:
            sentences = re.split(r'(?<=[.!?])\s+', text)
    else:
        sentences = re.split(r'(?<=[.!?])\s+', text)
    
    chunks = []
    current_chunk = []
    current_length = 0
    idx = 0
    current_start = 0
    
    for sentence in sentences:
        if current_length + len(sentence) > target_size and current_chunk:
            chunk_text = " ".join(current_chunk)
            chunks.append({"text": chunk_text, "chunk_index": idx, "start_char": current_start, "end_char": current_start + len(chunk_text), "strategy": "semantic"})
            current_start += len(chunk_text) + 1
            idx += 1
            current_chunk = []
            current_length = 0
        current_chunk.append(sentence)
        current_length += len(sentence) + 1
    
    if current_chunk:
        chunk_text = " ".join(current_chunk)
        chunks.append({"text": chunk_text, "chunk_index": idx, "start_char": current_start, "end_char": current_start + len(chunk_text), "strategy": "semantic"})
    return chunks

def recursive_chunking(text: str, max_chunk_size: int = 500) -> List[Dict]:
    if not text or len(text) <= max_chunk_size:
        return [{"text": text, "chunk_index": 0, "start_char": 0, "end_char": len(text), "strategy": "recursive"}] if text else []
    
    paragraphs = re.split(r'\n\s*\n', text)
    paragraphs = [p.strip() for p in paragraphs if p.strip()]
    
    chunks = []
    for para in paragraphs:
        if len(para) <= max_chunk_size:
            chunks.append({"text": para, "chunk_index": len(chunks), "start_char": 0, "end_char": len(para), "strategy": "recursive"})
        else:
            sub_chunks = fixed_length_chunking(para, max_chunk_size, 50)
            for sc in sub_chunks:
                sc["chunk_index"] = len(chunks)
                sc["strategy"] = "recursive"
                chunks.append(sc)
    return chunks

def get_chunks(text: str, strategy: str, **kwargs) -> List[Dict]:
    if strategy == "fixed":
        return fixed_length_chunking(text, **kwargs)
    elif strategy == "semantic":
        return semantic_chunking(text, **kwargs)
    elif strategy == "recursive":
        return recursive_chunking(text, **kwargs)
    return fixed_length_chunking(text, **kwargs)

# ChromaDB
_chroma_client = None

def get_chroma_client():
    global _chroma_client
    if _chroma_client is None:
        _chroma_client = chromadb.PersistentClient(path=str(CHROMA_DIR))
    return _chroma_client

def get_collection_name(model: str, strategy: str) -> str:
    return f"{model.replace('/', '_').replace('-', '_')}_{strategy}"

def initialize_chromadb(collection_name: str):
    client = get_chroma_client()
    return client.get_or_create_collection(name=collection_name, metadata={"hnsw:space": "cosine"})

def index_documents(chunks, embeddings, source_file, collection, embedding_model, chunking_strategy):
    if not chunks or not embeddings:
        return 0
    ids = []
    documents = []
    metadatas = []
    for i, (chunk, emb) in enumerate(zip(chunks, embeddings)):
        chunk_id = f"{source_file}_{chunking_strategy}_{i}"
        ids.append(chunk_id)
        documents.append(chunk["text"])
        metadatas.append({
            "source_file": source_file,
            "chunk_index": chunk.get("chunk_index", i),
            "chunking_strategy": chunking_strategy,
            "embedding_model": embedding_model,
            "indexed_at": datetime.now().isoformat()
        })
    collection.add(ids=ids, documents=documents, embeddings=embeddings, metadatas=metadatas)
    return len(ids)

def retrieve_similar_chunks(query, collection, embedding_model, api_key, top_k=5):
    query_embedding = get_embedding(query, embedding_model, api_key)
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k, include=["documents", "metadatas", "distances"])
    formatted = []
    if results and results['ids'] and len(results['ids']) > 0:
        for i in range(len(results['ids'][0])):
            formatted.append({
                "chunk_id": results['ids'][0][i],
                "text": results['documents'][0][i] if results['documents'] else "",
                "metadata": results['metadatas'][0][i] if results['metadatas'] else {},
                "distance": results['distances'][0][i] if results['distances'] else 0,
                "similarity": 1 - results['distances'][0][i] if results['distances'] else 1
            })
    return formatted

def list_collections():
    client = get_chroma_client()
    return [c.name for c in client.list_collections()]

def delete_collection(collection_name):
    try:
        client = get_chroma_client()
        client.delete_collection(collection_name)
        return True
    except:
        return False

def get_collection_stats(collection_name):
    try:
        client = get_chroma_client()
        collection = client.get_collection(collection_name)
        return {"name": collection_name, "count": collection.count()}
    except Exception as e:
        return {"name": collection_name, "error": str(e)}

# ============================================================
# Streamlit App
# ============================================================

st.set_page_config(
    page_title="RAG Resume Analysis",
    page_icon="📄",
    layout="wide"
)

# Sidebar navigation
st.sidebar.title("📄 RAG Resume Analysis")
page = st.sidebar.radio("Navigation", ["⚙️ Configuration & Upload", "📊 Evaluation & Testing"])

# Session state initialization
if "api_key" not in st.session_state:
    st.session_state.api_key = ""
if "api_valid" not in st.session_state:
    st.session_state.api_valid = False
if "indexed_stats" not in st.session_state:
    st.session_state.indexed_stats = None

# ============================================================
# Page 1: Configuration & Upload
# ============================================================
if page == "⚙️ Configuration & Upload":
    st.title("⚙️ Configuration & Upload")
    
    # API Key Section
    st.header("🔑 API Configuration")
    col1, col2 = st.columns([3, 1])
    with col1:
        api_key = st.text_input("OpenRouter API Key", type="password", value=st.session_state.api_key)
        st.session_state.api_key = api_key
    with col2:
        if st.button("Validate Key"):
            if api_key:
                with st.spinner("Validating..."):
                    is_valid, message = validate_openrouter_key(api_key)
                    st.session_state.api_valid = is_valid
                    if is_valid:
                        st.success(message)
                    else:
                        st.error(message)
            else:
                st.warning("Please enter an API key")
    
    if st.session_state.api_valid:
        st.success("✓ API Key validated")
    
    st.divider()
    
    # Model & Strategy Selection
    st.header("🤖 Model & Strategy Selection")
    col1, col2 = st.columns(2)
    with col1:
        selected_model = st.selectbox("Embedding Model", EMBEDDING_MODELS)
    with col2:
        selected_strategy = st.selectbox("Chunking Strategy", CHUNKING_STRATEGIES)
    
    # Chunking Parameters
    with st.expander("Advanced Chunking Parameters"):
        chunk_size = st.slider("Chunk Size (characters)", 100, 2000, DEFAULT_CHUNK_SIZE)
        overlap = st.slider("Overlap (characters)", 0, 200, DEFAULT_OVERLAP)
    
    st.divider()
    
    # File Upload Section
    st.header("📁 Resume Upload")
    
    # Option 1: Upload files
    uploaded_files = st.file_uploader("Upload PDF Resumes", type=["pdf"], accept_multiple_files=True)
    
    # Option 2: Use files from data/ folder
    existing_pdfs = list(DATA_DIR.glob("*.pdf"))
    if existing_pdfs:
        st.info(f"📂 Found {len(existing_pdfs)} PDF files in data/ folder")
        use_existing = st.checkbox("Use existing PDFs from data/ folder", value=True)
    else:
        use_existing = False
        st.info("💡 Add PDF files to the 'data/' folder or upload them above")
    
    st.divider()
    
    # Process Button
    st.header("🚀 Process & Index")
    
    if st.button("Process & Index Resumes", type="primary", disabled=not st.session_state.api_valid):
        pdf_paths = []
        
        # Save uploaded files
        if uploaded_files:
            for uploaded_file in uploaded_files:
                file_path = DATA_DIR / uploaded_file.name
                with open(file_path, "wb") as f:
                    f.write(uploaded_file.getbuffer())
                pdf_paths.append(file_path)
        
        # Add existing PDFs
        if use_existing and existing_pdfs:
            pdf_paths.extend(existing_pdfs)
        
        # Remove duplicates
        pdf_paths = list(set(pdf_paths))
        
        if not pdf_paths:
            st.error("No PDF files to process!")
        else:
            collection_name = get_collection_name(selected_model, selected_strategy)
            
            progress_bar = st.progress(0)
            status_text = st.empty()
            
            total_chunks = 0
            processed = 0
            
            for i, pdf_path in enumerate(pdf_paths):
                status_text.text(f"Processing: {pdf_path.name}")
                
                # Extract text
                text = extract_text_from_pdf(str(pdf_path))
                if not text:
                    continue
                
                # Chunk
                chunks = get_chunks(text, selected_strategy, chunk_size=chunk_size, overlap=overlap)
                if not chunks:
                    continue
                
                # Get embeddings
                chunk_texts = [c["text"] for c in chunks]
                embeddings = []
                for ct in chunk_texts:
                    emb = get_embedding(ct, selected_model, st.session_state.api_key)
                    embeddings.append(emb)
                
                # Index
                collection = initialize_chromadb(collection_name)
                indexed = index_documents(chunks, embeddings, pdf_path.name, collection, selected_model, selected_strategy)
                
                total_chunks += indexed
                processed += 1
                progress_bar.progress((i + 1) / len(pdf_paths))
            
            status_text.text("Complete!")
            
            st.session_state.indexed_stats = {
                "collection_name": collection_name,
                "files_processed": processed,
                "total_chunks": total_chunks,
                "model": selected_model,
                "strategy": selected_strategy
            }
            
            st.success(f"✅ Indexed {total_chunks} chunks from {processed} files into collection: {collection_name}")
    
    # Show indexed stats
    if st.session_state.indexed_stats:
        st.divider()
        st.subheader("📊 Last Indexing Summary")
        stats = st.session_state.indexed_stats
        col1, col2, col3 = st.columns(3)
        col1.metric("Files Processed", stats["files_processed"])
        col2.metric("Total Chunks", stats["total_chunks"])
        col3.metric("Collection", stats["collection_name"][:30] + "...")
    
    # Collection Management
    st.divider()
    st.header("🗄️ Collection Management")
    
    collections = list_collections()
    if collections:
        st.write(f"**Existing Collections:** {len(collections)}")
        
        for coll_name in collections:
            stats = get_collection_stats(coll_name)
            col1, col2, col3 = st.columns([3, 1, 1])
            col1.write(f"📁 {coll_name}")
            col2.write(f"Chunks: {stats.get('count', 'N/A')}")
            if col3.button("Delete", key=f"del_{coll_name}"):
                if delete_collection(coll_name):
                    st.success(f"Deleted {coll_name}")
                    st.rerun()
    else:
        st.info("No collections found. Process some resumes first!")

# ============================================================
# Page 2: Evaluation & Testing
# ============================================================
else:
    st.title("📊 Evaluation & Testing")
    
    if not st.session_state.api_valid:
        st.warning("⚠️ Please validate your API key on the Configuration page first!")
        st.stop()
    
    # Collection Selection
    collections = list_collections()
    if not collections:
        st.warning("No collections found. Please index some resumes first!")
        st.stop()
    
    st.header("🎯 Query Configuration")
    
    col1, col2 = st.columns(2)
    with col1:
        selected_collection = st.selectbox("Select Collection", collections)
    with col2:
        top_k = st.number_input("Top-K Results", min_value=1, max_value=20, value=DEFAULT_TOP_K)
    
    # Parse collection name to get model
    collection_parts = selected_collection.rsplit("_", 1)
    if len(collection_parts) == 2:
        model_part = collection_parts[0].replace("_", "/").replace("openai/", "openai/")
        # Find matching model
        eval_model = None
        for m in EMBEDDING_MODELS:
            if m.replace("/", "_").replace("-", "_") in selected_collection:
                eval_model = m
                break
        if not eval_model:
            eval_model = EMBEDDING_MODELS[0]
    else:
        eval_model = EMBEDDING_MODELS[0]
    
    st.info(f"Using model: {eval_model}")
    
    st.divider()
    
    # Query Input
    st.header("🔍 Evaluation Queries")
    
    query_input = st.text_area(
        "Enter queries (one per line)",
        height=150,
        placeholder="What is the candidate's email?\nWhat programming languages does the candidate know?\nWhat is their education background?"
    )
    
    # Ground Truth (optional)
    with st.expander("📋 Ground Truth (Optional)"):
        st.write("Upload or paste ground truth JSON for accurate metrics calculation")
        gt_json = st.text_area("Ground Truth JSON", height=100, placeholder='{"q_0": {"relevant": ["answer1", "answer2"]}}')
        ground_truth = {}
        if gt_json:
            try:
                ground_truth = json.loads(gt_json)
            except:
                st.error("Invalid JSON format")
    
    st.divider()
    
    # Run Evaluation
    if st.button("🚀 Run Evaluation", type="primary"):
        queries = [q.strip() for q in query_input.strip().split("\n") if q.strip()]
        
        if not queries:
            st.error("Please enter at least one query")
        else:
            collection = initialize_chromadb(selected_collection)
            
            results = []
            all_metrics = []
            
            progress = st.progress(0)
            
            for i, query in enumerate(queries):
                start_time = time.time()
                retrieved = retrieve_similar_chunks(query, collection, eval_model, st.session_state.api_key, top_k)
                latency = (time.time() - start_time) * 1000
                
                # Calculate metrics
                retrieved_ids = [r["chunk_id"] for r in retrieved]
                relevant = ground_truth.get(f"q_{i}", {}).get("relevant", [])
                
                if relevant:
                    precision = len(set(retrieved_ids) & set(relevant)) / len(retrieved_ids) if retrieved_ids else 0
                    recall = len(set(retrieved_ids) & set(relevant)) / len(relevant) if relevant else 0
                    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
                else:
                    precision = recall = f1 = 0
                
                avg_sim = np.mean([r["similarity"] for r in retrieved]) if retrieved else 0
                
                all_metrics.append({
                    "Query": query[:50] + "..." if len(query) > 50 else query,
                    "Precision": precision,
                    "Recall": recall,
                    "F1": f1,
                    "Avg Similarity": avg_sim,
                    "Latency (ms)": latency
                })
                
                results.append({"query": query, "results": retrieved, "metrics": all_metrics[-1]})
                progress.progress((i + 1) / len(queries))
            
            # Display Results
            st.header("📊 Results")
            
            # Metrics Summary
            metrics_df = pd.DataFrame(all_metrics)
            
            col1, col2, col3, col4 = st.columns(4)
            col1.metric("Avg Precision", f"{metrics_df['Precision'].mean():.3f}")
            col2.metric("Avg Recall", f"{metrics_df['Recall'].mean():.3f}")
            col3.metric("Avg F1", f"{metrics_df['F1'].mean():.3f}")
            col4.metric("Avg Latency", f"{metrics_df['Latency (ms)'].mean():.1f} ms")
            
            st.divider()
            
            # Metrics Table
            st.subheader("📋 Per-Query Metrics")
            st.dataframe(metrics_df, use_container_width=True)
            
            # Similarity Chart
            if len(metrics_df) > 0:
                fig = px.bar(metrics_df, x="Query", y="Avg Similarity", title="Context Relevance by Query")
                st.plotly_chart(fig, use_container_width=True)
            
            st.divider()
            
            # Retrieved Chunks
            st.subheader("🔍 Retrieved Chunks")
            
            for r in results:
                with st.expander(f"Query: {r['query'][:80]}..."):
                    for j, chunk in enumerate(r["results"]):
                        st.markdown(f"**Rank {j+1}** (Similarity: {chunk['similarity']:.3f})")
                        st.text(chunk["text"][:500] + "..." if len(chunk["text"]) > 500 else chunk["text"])
                        st.caption(f"Source: {chunk['metadata'].get('source_file', 'Unknown')}")
                        st.divider()
            
            # Export
            st.divider()
            st.subheader("💾 Export Results")
            
            col1, col2 = st.columns(2)
            
            with col1:
                csv_data = metrics_df.to_csv(index=False)
                st.download_button("📥 Download CSV", csv_data, "evaluation_results.csv", "text/csv")
            
            with col2:
                json_data = json.dumps({"metrics": all_metrics, "results": [{"query": r["query"], "retrieved_count": len(r["results"])} for r in results]}, indent=2)
                st.download_button("📥 Download JSON", json_data, "evaluation_results.json", "application/json")
    
    # A/B Comparison
    st.divider()
    st.header("🔬 A/B Comparison")
    
    if len(collections) > 1:
        st.write("Compare multiple model+strategy configurations:")
        
        selected_for_comparison = st.multiselect("Select collections to compare", collections, default=collections[:2] if len(collections) >= 2 else collections)
        
        if st.button("Compare Selected") and selected_for_comparison and query_input:
            queries = [q.strip() for q in query_input.strip().split("\n") if q.strip()]
            
            comparison_results = []
            
            for coll_name in selected_for_comparison:
                collection = initialize_chromadb(coll_name)
                
                # Find model for this collection
                coll_model = eval_model
                for m in EMBEDDING_MODELS:
                    if m.replace("/", "_").replace("-", "_") in coll_name:
                        coll_model = m
                        break
                
                total_sim = 0
                total_latency = 0
                
                for query in queries:
                    start = time.time()
                    retrieved = retrieve_similar_chunks(query, collection, coll_model, st.session_state.api_key, top_k)
                    latency = (time.time() - start) * 1000
                    
                    total_sim += np.mean([r["similarity"] for r in retrieved]) if retrieved else 0
                    total_latency += latency
                
                comparison_results.append({
                    "Collection": coll_name,
                    "Avg Similarity": total_sim / len(queries),
                    "Avg Latency (ms)": total_latency / len(queries)
                })
            
            comparison_df = pd.DataFrame(comparison_results)
            st.dataframe(comparison_df, use_container_width=True)
            
            # Chart
            fig = px.bar(comparison_df, x="Collection", y="Avg Similarity", title="Comparison: Average Context Relevance")
            st.plotly_chart(fig, use_container_width=True)
    else:
        st.info("Index resumes with different models/strategies to enable comparison")

Writing app.py


## 6. Sample Data Files

Run the cells below to create sample metadata and ground truth files.

In [15]:
# Create sample metadata.csv
metadata_content = """source_file,candidate_id,candidate_name,emails,phones,universities,education_timeline,skills,projects,notable_keywords
resume1.pdf,cand_001,John Doe,john.doe@email.com,+1-555-0101,MIT,BS Computer Science|MIT|2018|2022,Python;JavaScript;React;Node.js;SQL,E-commerce Platform;ML Pipeline,machine learning;web development
resume2.pdf,cand_002,Jane Smith,jane.smith@email.com;jsmith@work.com,+1-555-0102,Stanford University,MS Data Science|Stanford|2020|2022;BS Mathematics|UCLA|2016|2020,Python;R;TensorFlow;PyTorch;SQL;Tableau,Fraud Detection System;Customer Segmentation,data science;analytics
resume3.pdf,cand_003,Alex Johnson,alex.j@email.com,,Harvard University,MBA|Harvard|2019|2021;BS Finance|NYU|2013|2017,Excel;PowerBI;SQL;Python;Financial Modeling,Investment Analysis Tool;Risk Assessment Dashboard,finance;business intelligence
"""

metadata_path = DATA_DIR / "metadata.csv"
with open(metadata_path, 'w', encoding='utf-8') as f:
    f.write(metadata_content)

print(f"✅ Created sample metadata file: {metadata_path}")

✅ Created sample metadata file: data\metadata.csv


In [16]:
# Create sample ground_truth.json
ground_truth_content = {
    "q_0": {
        "query_text": "What is the candidate's email address?",
        "relevant": [
            {"source_file": "resume1.pdf", "candidate_id": "cand_001", "type": "exact_answer", "value": "john.doe@email.com"},
            {"source_file": "resume2.pdf", "candidate_id": "cand_002", "type": "exact_answer", "value": "jane.smith@email.com"}
        ]
    },
    "q_1": {
        "query_text": "What programming languages does the candidate know?",
        "relevant": [
            {"source_file": "resume1.pdf", "candidate_id": "cand_001", "type": "partial_answer", "value": "Python, JavaScript"},
            {"source_file": "resume2.pdf", "candidate_id": "cand_002", "type": "partial_answer", "value": "Python, R"}
        ]
    },
    "q_2": {
        "query_text": "What is the candidate's educational background?",
        "relevant": [
            {"source_file": "resume1.pdf", "candidate_id": "cand_001", "type": "partial_answer", "value": "BS Computer Science from MIT"},
            {"source_file": "resume2.pdf", "candidate_id": "cand_002", "type": "partial_answer", "value": "MS Data Science from Stanford, BS Mathematics from UCLA"},
            {"source_file": "resume3.pdf", "candidate_id": "cand_003", "type": "partial_answer", "value": "MBA from Harvard, BS Finance from NYU"}
        ]
    },
    "q_3": {
        "query_text": "What projects has the candidate worked on?",
        "relevant": [
            {"source_file": "resume1.pdf", "candidate_id": "cand_001", "type": "partial_answer", "value": "E-commerce Platform, ML Pipeline"},
            {"source_file": "resume2.pdf", "candidate_id": "cand_002", "type": "partial_answer", "value": "Fraud Detection System, Customer Segmentation"}
        ]
    },
    "q_4": {
        "query_text": "Does the candidate have machine learning experience?",
        "relevant": [
            {"source_file": "resume1.pdf", "candidate_id": "cand_001", "type": "boolean_answer", "value": "yes"},
            {"source_file": "resume2.pdf", "candidate_id": "cand_002", "type": "boolean_answer", "value": "yes"}
        ]
    }
}

ground_truth_path = DATA_DIR / "ground_truth.json"
with open(ground_truth_path, 'w', encoding='utf-8') as f:
    json.dump(ground_truth_content, f, indent=2)

print(f"✅ Created sample ground truth file: {ground_truth_path}")
print(f"\n📋 Ground truth contains {len(ground_truth_content)} sample queries")

✅ Created sample ground truth file: data\ground_truth.json

📋 Ground truth contains 5 sample queries


## 7. Test Implementation

Quick tests to verify all modules work correctly.

In [17]:
# ============================================================
# Test: Chunking Strategies
# ============================================================

sample_text = """
John Doe
Software Engineer

Contact Information:
Email: john.doe@email.com
Phone: +1-555-0101
LinkedIn: linkedin.com/in/johndoe

Summary:
Experienced software engineer with 5+ years of expertise in full-stack development.
Proficient in Python, JavaScript, and cloud technologies. Passionate about building
scalable applications and mentoring junior developers.

Education:
Bachelor of Science in Computer Science
Massachusetts Institute of Technology (MIT)
Graduated: May 2022
GPA: 3.8/4.0

Experience:
Senior Software Engineer - TechCorp Inc.
January 2022 - Present
- Led development of microservices architecture serving 1M+ users
- Implemented CI/CD pipelines reducing deployment time by 60%
- Mentored team of 5 junior developers

Skills:
Programming: Python, JavaScript, TypeScript, Java, SQL
Frameworks: React, Node.js, Django, FastAPI
Cloud: AWS, GCP, Docker, Kubernetes
Tools: Git, Jenkins, Terraform

Projects:
1. E-commerce Platform - Built scalable shopping platform with React and Node.js
2. ML Pipeline - Developed automated machine learning pipeline for predictions
"""

print("🧪 Testing Chunking Strategies\n")
print("=" * 60)

# Test Fixed Chunking
fixed_chunks = fixed_length_chunking(sample_text, chunk_size=300, overlap=50)
print(f"\n📦 Fixed Chunking (300 chars, 50 overlap):")
print(f"   Chunks created: {len(fixed_chunks)}")
print(f"   First chunk length: {len(fixed_chunks[0]['text'])} chars")

# Test Semantic Chunking
semantic_chunks = semantic_chunking(sample_text, target_size=300)
print(f"\n📦 Semantic Chunking (target 300 chars):")
print(f"   Chunks created: {len(semantic_chunks)}")
print(f"   First chunk length: {len(semantic_chunks[0]['text'])} chars")

# Test Recursive Chunking
recursive_chunks = recursive_chunking(sample_text, max_chunk_size=300)
print(f"\n📦 Recursive Chunking (max 300 chars):")
print(f"   Chunks created: {len(recursive_chunks)}")
print(f"   First chunk length: {len(recursive_chunks[0]['text'])} chars")

print("\n" + "=" * 60)
print("✅ All chunking strategies working correctly!")

🧪 Testing Chunking Strategies


📦 Fixed Chunking (300 chars, 50 overlap):
   Chunks created: 5
   First chunk length: 300 chars

📦 Semantic Chunking (target 300 chars):
   Chunks created: 4
   First chunk length: 280 chars

📦 Recursive Chunking (max 300 chars):
   Chunks created: 7
   First chunk length: 26 chars

✅ All chunking strategies working correctly!


In [18]:
# ============================================================
# Test: Evaluation Metrics
# ============================================================

print("🧪 Testing Evaluation Metrics\n")
print("=" * 60)

# Test data
retrieved = ["chunk_1", "chunk_2", "chunk_3", "chunk_4", "chunk_5"]
relevant = ["chunk_1", "chunk_3", "chunk_6", "chunk_7"]

precision = calculate_precision(retrieved, relevant)
recall = calculate_recall(retrieved, relevant)
f1 = calculate_f1_score(precision, recall)

print(f"\n📊 Test Case:")
print(f"   Retrieved: {retrieved}")
print(f"   Relevant:  {relevant}")
print(f"\n   Precision: {precision:.3f} (expected: 0.400)")
print(f"   Recall:    {recall:.3f} (expected: 0.500)")
print(f"   F1 Score:  {f1:.3f} (expected: 0.444)")

# Test MRR
ranked_results = [["chunk_5", "chunk_1", "chunk_3"], ["chunk_2", "chunk_6"]]
mrr_relevant = ["chunk_1", "chunk_6"]
mrr = calculate_mrr(ranked_results, mrr_relevant)
print(f"\n   MRR: {mrr:.3f} (first hit at rank 2 and 2)")

print("\n" + "=" * 60)
print("✅ All evaluation metrics working correctly!")

🧪 Testing Evaluation Metrics


📊 Test Case:
   Retrieved: ['chunk_1', 'chunk_2', 'chunk_3', 'chunk_4', 'chunk_5']
   Relevant:  ['chunk_1', 'chunk_3', 'chunk_6', 'chunk_7']

   Precision: 0.400 (expected: 0.400)
   Recall:    0.500 (expected: 0.500)
   F1 Score:  0.444 (expected: 0.444)

   MRR: 0.500 (first hit at rank 2 and 2)

✅ All evaluation metrics working correctly!


In [19]:
# ============================================================
# Test: Ground Truth Loading
# ============================================================

print("🧪 Testing Ground Truth Functions\n")
print("=" * 60)

# Load the sample files we created
gt = load_ground_truth()
metadata = load_metadata()

print(f"\n📋 Ground Truth loaded:")
print(f"   Number of queries: {len(gt)}")
if gt:
    first_key = list(gt.keys())[0]
    print(f"   Sample query: {gt[first_key].get('query_text', 'N/A')[:50]}...")

print(f"\n📋 Metadata loaded:")
print(f"   Number of entries: {len(metadata)}")
if not metadata.empty:
    print(f"   Columns: {list(metadata.columns)}")

print("\n" + "=" * 60)
print("✅ Ground truth functions working correctly!")

🧪 Testing Ground Truth Functions


📋 Ground Truth loaded:
   Number of queries: 5
   Sample query: What is the candidate's email address?...

📋 Metadata loaded:
   Number of entries: 3
   Columns: ['source_file', 'candidate_id', 'candidate_name', 'emails', 'phones', 'universities', 'education_timeline', 'skills', 'projects', 'notable_keywords']

✅ Ground truth functions working correctly!


## 8. Quick Start Guide

### To use this RAG Resume Analysis system:

1. **Add your OpenRouter API key** (get one at https://openrouter.ai)

2. **Add resume PDFs** to the `data/` folder

3. **Run all cells** in this notebook to set up the functions

4. **Launch Streamlit app**:
   ```bash
   streamlit run app.py
   ```

5. **In the Streamlit UI**:
   - Page 1: Enter API key, select model/strategy, upload or use existing PDFs, click "Process & Index"
   - Page 2: Enter queries, run evaluation, view metrics, export results

### Alternative: Use notebook directly

You can also use the functions directly in this notebook:

In [20]:
# ============================================================
# Example: Using the notebook directly
# ============================================================

# Uncomment and fill in your API key to test:
# API_KEY = "your-openrouter-api-key-here"

# Example workflow:
"""
# 1. Validate API key
is_valid, msg = validate_openrouter_key(API_KEY)
print(msg)

# 2. Get PDF files
pdf_files = get_pdf_files(DATA_DIR)

# 3. Process and index resumes
stats = process_resumes(
    pdf_files=pdf_files,
    chunking_strategy="semantic",
    embedding_model="openai/text-embedding-3-small",
    api_key=API_KEY
)
print(stats)

# 4. Run evaluation queries
queries = [
    "What is the candidate's email address?",
    "What programming languages does the candidate know?",
    "What is their education background?"
]

results = evaluate_retrieval(
    queries=queries,
    collection_name=stats["collection_name"],
    embedding_model="openai/text-embedding-3-small",
    api_key=API_KEY,
    top_k=5
)

# 5. Export results
export_results_csv(results)
export_results_json(results)
"""

print("💡 Uncomment the code above and add your API key to test the full pipeline!")
print("\n📱 Or run: streamlit run app.py")

💡 Uncomment the code above and add your API key to test the full pipeline!

📱 Or run: streamlit run app.py
